In [1]:
!pip install -q transformers datasets accelerate peft bitsandbytes tqdm numpy evaluate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.8 MB/s eta 0:00:00


In [2]:
# !pip install -q lime shap

In [4]:
from huggingface_hub import login
login()

In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import AutoModelForCausalLM
from peft import PeftModelForCausalLM, PeftModel, PeftConfig
import torch
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf

import re
from itertools import combinations
import json
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [6]:
input_prompt = """
Using the context below, estimate the rate of change in the closing price of SK하이닉스 on 2025-05-24.
Return the expected value of change as a decimal.

Context: date, open, high, low, close, volume, change.
2025-04-28, 55800, 55900, 55500, 55800, 9341518, 0.0000000000000000
2025-04-29, 55700, 56500, 55600, 55800, 10295518, 0.0000000000000000
2025-04-30, 55900, 56000, 55300, 55500, 13014189, -0.0053763440860215
2025-05-02, 55000, 55500, 54200, 54300, 22454204, -0.0216216216216216
2025-05-07, 54500, 55000, 54300, 54600, 17136991, 0.0055248618784531
2025-05-08, 55100, 55500, 54500, 54600, 16320532, 0.0000000000000000
2025-05-09, 54700, 55000, 54400, 54800, 7814322, 0.0036630036630036

Answer:
"""


In [7]:
def extract_context_lines(input_prompt: str):
    lines = input_prompt.strip().split('\n')
    context_start = next(i for i, line in enumerate(lines) if line.startswith("Context:"))
    data_lines = lines[context_start + 1:]
    return data_lines  # 각 날짜별 CSV 줄 리스트


In [8]:
context_lines = extract_context_lines(input_prompt)
for i, line in enumerate(context_lines):
    print(f"{i}: {line}")


0: 2025-04-28, 55800, 55900, 55500, 55800, 9341518, 0.0000000000000000
1: 2025-04-29, 55700, 56500, 55600, 55800, 10295518, 0.0000000000000000
2: 2025-04-30, 55900, 56000, 55300, 55500, 13014189, -0.0053763440860215
3: 2025-05-02, 55000, 55500, 54200, 54300, 22454204, -0.0216216216216216
4: 2025-05-07, 54500, 55000, 54300, 54600, 17136991, 0.0055248618784531
5: 2025-05-08, 55100, 55500, 54500, 54600, 16320532, 0.0000000000000000
6: 2025-05-09, 54700, 55000, 54400, 54800, 7814322, 0.0036630036630036
7: 
8: Answer:


In [9]:
def make_masked_prompt(context_lines, masked_indices):
    masked_context = [line for i, line in enumerate(context_lines) if i not in masked_indices]
    new_context = "Context: date, open, high, low, close, volume, change.\n" + "\n".join(masked_context)
    new_prompt = (
        "Using the context below, estimate the rate of change in the closing price of SK하이닉스 on 2025-05-24.\n"
        "Return the expected value of change as a decimal.\n\n"
        + new_context + "\n\nAnswer:"
    )
    return new_prompt


In [10]:
masked_prompt = make_masked_prompt(context_lines, masked_indices=[4])
print(masked_prompt)


Using the context below, estimate the rate of change in the closing price of SK하이닉스 on 2025-05-24.
Return the expected value of change as a decimal.

Context: date, open, high, low, close, volume, change.
2025-04-28, 55800, 55900, 55500, 55800, 9341518, 0.0000000000000000
2025-04-29, 55700, 56500, 55600, 55800, 10295518, 0.0000000000000000
2025-04-30, 55900, 56000, 55300, 55500, 13014189, -0.0053763440860215
2025-05-02, 55000, 55500, 54200, 54300, 22454204, -0.0216216216216216
2025-05-08, 55100, 55500, 54500, 54600, 16320532, 0.0000000000000000
2025-05-09, 54700, 55000, 54400, 54800, 7814322, 0.0036630036630036

Answer:

Answer:


In [11]:
from itertools import combinations

def generate_masked_prompts(input_prompt, max_mask_count=2):
    """
    input_prompt: 원본 프롬프트 전체 (str)
    max_mask_count: 한 번에 제거할 최대 줄 수 (1: 단일, 2: 이중 마스킹까지 생성)
    return: (masked_prompt_str, masked_indices) 리스트
    """
    context_lines = extract_context_lines(input_prompt)
    total_rows = len(context_lines)
    all_masked_prompts = []

    for r in range(1, max_mask_count + 1):  # 1개, 2개 제거 조합
        for masked_indices in combinations(range(total_rows), r):
            masked_prompt = make_masked_prompt(context_lines, masked_indices)
            all_masked_prompts.append((masked_prompt, masked_indices))

    return all_masked_prompts


In [12]:
all_prompts = generate_masked_prompts(input_prompt, max_mask_count=2)

# 몇 개만 출력해보자
for i, (prompt, indices) in enumerate(all_prompts[:3]):
    print(f"---- Masked Indices: {indices} ----")
    print(prompt)
    print("\n")


---- Masked Indices: (0,) ----
Using the context below, estimate the rate of change in the closing price of SK하이닉스 on 2025-05-24.
Return the expected value of change as a decimal.

Context: date, open, high, low, close, volume, change.
2025-04-29, 55700, 56500, 55600, 55800, 10295518, 0.0000000000000000
2025-04-30, 55900, 56000, 55300, 55500, 13014189, -0.0053763440860215
2025-05-02, 55000, 55500, 54200, 54300, 22454204, -0.0216216216216216
2025-05-07, 54500, 55000, 54300, 54600, 17136991, 0.0055248618784531
2025-05-08, 55100, 55500, 54500, 54600, 16320532, 0.0000000000000000
2025-05-09, 54700, 55000, 54400, 54800, 7814322, 0.0036630036630036

Answer:

Answer:


---- Masked Indices: (1,) ----
Using the context below, estimate the rate of change in the closing price of SK하이닉스 on 2025-05-24.
Return the expected value of change as a decimal.

Context: date, open, high, low, close, volume, change.
2025-04-28, 55800, 55900, 55500, 55800, 9341518, 0.0000000000000000
2025-04-30, 55900, 56000,

In [13]:
import json

with open("masked_prompts.jsonl", "w", encoding="utf-8") as f:
    for prompt, mask_idx in all_prompts:
        f.write(json.dumps({
            "masked_indices": mask_idx,
            "masked_prompt": prompt
        }, ensure_ascii=False) + "\n")


# Step 3

| 항목    | 내용                                                |
| ----- | ------------------------------------------------- |
| 🎯 목표 | 마스킹된 프롬프트들 각각에 대해 LLM(FinMA)을 실행해서 예측값 `f(xₛ)` 얻기 |
| ⚖️ 비교 | 원본 프롬프트 예측값 `f(x)`과 비교 → `Δ = f(x) - f(xₛ)`       |
| 📦 결과 | FastSHAP surrogate 학습을 위한 `(mask, Δ)` 샘플 생성       |


In [14]:
# f(x) = FinMA(input_prompt)  # 원본 전체 프롬프트
# f(x_s) = FinMA(masked_prompt)  # 특정 날짜 줄이 제거된 프롬프트
# delta = f(x) - f(x_s)


In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


In [16]:
from transformers import BitsAndBytesConfig

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForCausalLM.from_pretrained(
    "capston-team-5/finma-7b-4bit-quantized",
    device_map="auto",  # GPU/CPU 자동 할당
    quantization_config=bnb_cfg
)

model = PeftModel.from_pretrained(
    base_model,
    "capston-team-5/finma-7b-lora-regression-v1"
)

tokenizer = AutoTokenizer.from_pretrained("capston-team-5/finma-7b-lora-regression-v1")

model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:609: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

In [17]:
import re

def get_finma_prediction(prompt, model, tokenizer):
    """
    FinMA 모델에 프롬프트를 넣고 예측값을 추출 (float 한 개 반환)
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # float 숫자만 정규식으로 추출
    matches = re.findall(r"[-+]?\d*\.\d+|\d+", decoded)
    if matches:
        try:
            return float(matches[-1])  # 가장 마지막 실수 반환
        except:
            pass
    raise ValueError("예측값 추출 실패: " + decoded)


In [18]:

# 예측값 출력
print(get_finma_prediction(prompt, model, tokenizer))

2.0


In [19]:
prompt

'Using the context below, estimate the rate of change in the closing price of SK하이닉스 on 2025-05-24.\nReturn the expected value of change as a decimal.\n\nContext: date, open, high, low, close, volume, change.\n2025-04-28, 55800, 55900, 55500, 55800, 9341518, 0.0000000000000000\n2025-04-29, 55700, 56500, 55600, 55800, 10295518, 0.0000000000000000\n2025-04-30, 55900, 56000, 55300, 55500, 13014189, -0.0053763440860215\n2025-05-02, 55000, 55500, 54200, 54300, 22454204, -0.0216216216216216\n2025-05-07, 54500, 55000, 54300, 54600, 17136991, 0.0055248618784531\n2025-05-08, 55100, 55500, 54500, 54600, 16320532, 0.0000000000000000\n2025-05-09, 54700, 55000, 54400, 54800, 7814322, 0.0036630036630036\n\nAnswer:'

In [20]:
original_pred = get_finma_prediction(input_prompt, model, tokenizer)

In [25]:
temp_pred = get_finma_prediction(prompt, model, tokenizer)
temp_pred

2.0

In [22]:
original_pred

2.0

In [21]:


results = []
for masked_prompt, mask_indices in all_prompts:
    masked_pred = get_finma_prediction(masked_prompt, model, tokenizer)
    delta = round(original_pred - masked_pred, 6)
    results.append({
        "mask_indices": mask_indices,
        "f_x": original_pred,
        "f_xs": masked_pred,
        "delta": delta
    })


In [23]:
for r in results[:5]:
    print(f"Removed rows: {r['mask_indices']} | f(x)={r['f_x']:.4f} | f(x_s)={r['f_xs']:.4f} | Δ={r['delta']:.4f}")


Removed rows: (0,) | f(x)=2.0000 | f(x_s)=0.0037 | Δ=1.9963
Removed rows: (1,) | f(x)=2.0000 | f(x_s)=0.0037 | Δ=1.9963
Removed rows: (2,) | f(x)=2.0000 | f(x_s)=0.0037 | Δ=1.9963
Removed rows: (3,) | f(x)=2.0000 | f(x_s)=0.0037 | Δ=1.9963
Removed rows: (4,) | f(x)=2.0000 | f(x_s)=0.0037 | Δ=1.9963


In [24]:
import json

with open("delta_results.jsonl", "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")


# Step 4

| 항목    | 내용                                                   |
| ----- | ---------------------------------------------------- |
| 🎯 목표 | 마스킹 벡터 `s`와 FinMA 예측 변화량 `Δ = f(x) - f(xₛ)` 쌍을 만들기   |
| 🧠 목적 | 이걸 기반으로 surrogate 모델이 각 feature(날짜 row)의 기여도를 추론하게 함 |
| 📦 출력 | `X_train` = 마스킹 벡터, `Y_train` = Δ 값 리스트              |


In [26]:
import numpy as np

# Δ 및 마스킹 벡터 수집

n_rows = len(extract_context_lines(input_prompt))
X_train = []
Y_train = []

for r in results:  # Step 3에서 저장한 리스트
    mask_vector = [1 if i in r['mask_indices'] else 0 for i in range(n_rows)]
    X_train.append(mask_vector)
    Y_train.append(r['delta'])


In [27]:
for x, y in zip(X_train[:5], Y_train[:5]):
    print(f"mask: {x}, delta: {y}")


mask: [1, 0, 0, 0, 0, 0, 0, 0, 0], delta: 1.996344
mask: [0, 1, 0, 0, 0, 0, 0, 0, 0], delta: 1.996344
mask: [0, 0, 1, 0, 0, 0, 0, 0, 0], delta: 1.996344
mask: [0, 0, 0, 1, 0, 0, 0, 0, 0], delta: 1.996344
mask: [0, 0, 0, 0, 1, 0, 0, 0, 0], delta: 1.996344


In [28]:
# 학습용 Tensor로 변환 (선택)

In [29]:
import torch

X_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_tensor = torch.tensor(Y_train, dtype=torch.float32).unsqueeze(1)


In [30]:
# 저장하기 (선택)

import pandas as pd

df = pd.DataFrame(X_train, columns=[f"row_{i}" for i in range(n_rows)])
df["delta"] = Y_train
df.to_csv("fastshap_training_data.csv", index=False)


# Step 5 : Train Surrogate

In [32]:
def train_surrogate(results):
    X = [r["mask"] for r in results]
    y = [r["delta"] for r in results]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = Ridge()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Surrogate Test MSE: {mse:.6f}")
    return model

# Step 6 : feature importance 시각화

In [33]:
def plot_feature_importance(surrogate_model):
    coefs = surrogate_model.coef_
    plt.figure(figsize=(10, 4))
    plt.bar(range(len(coefs)), coefs)
    plt.xlabel("Row index (date line)")
    plt.ylabel("Estimated SHAP value (importance)")
    plt.title("Surrogate Feature Importance")
    plt.grid(True)
    plt.show()

# 실행 및 시각화

In [34]:
surrogate = train_surrogate(results)
plot_feature_importance(surrogate)

KeyError: 'mask'